# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est décrit en détail au début du chapitre 4. Pour rappel, il est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé `telecom.csv`, accessible dans le répertoire `data`. On le charge en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn = pd.read_csv("./data/telecom.csv")
# on affiche les 5 premiers individus (lignes)
churn.head(5)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [3]:
# on affiche le type de chaque variable (colonne)
churn.dtypes

State              object
Account Length      int64
Area Code           int64
Phone              object
Int'l Plan         object
VMail Plan         object
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls          int64
Intl Charge       float64
CustServ Calls      int64
Churn?             object
dtype: object

## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de cinq colonnes `object`.

Nous pouvons afficher les statistiques descriptives pour les colonnes `object` :

In [4]:
# churn.describe().transpose()
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,410-9633,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les trois dernières variables `object` sont binaires.

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables.

Par ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [5]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

In [6]:
# !pip install pandas_profiling
import pandas_profiling
churn.profile_report(style={'full_width':True})

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Imputer
#from sklearn.impute import SimpleImputer

LabelEncoder va nous permettre de transformer les valeurs textuelles en entiers.
Nous pouvons utiliser pour chaque variable qualitative :

In [8]:
# création d'un objet
encoder = LabelEncoder()
# on applique les données à l'objet
churn["Churn?"] = encoder.fit_transform(churn["Churn?"])
orig_phone = churn["Phone"]

In [9]:
dict_label_encode = {}
for col in churn.columns:
    if churn[col].dtype == object:
        dict_label_encode[col] = LabelEncoder()
        churn[col] = dict_label_encode[col].fit_transform(churn[col])
churn.head(5)

,State,Account_Length,Area_Code,Phone,Int'l_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,...,Eve_Calls,Eve_Charge,Night_Mins,Night_Calls,Night_Charge,Intl_Mins,Intl_Calls,Intl_Charge,CustServ_Calls,Churn?
0,16,128,415,1926,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,35,107,415,1575,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,31,137,415,1117,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,35,84,408,1707,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,36,75,415,110,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [10]:
dict_label_encode["Phone"].__dict__
# vs OneHotEncoder
oh_enc = OneHotEncoder()
oh_phone = oh_enc.fit_transform([orig_phone])
oh_phone

<1x3333 sparse matrix of type '<class 'numpy.float64'>'
	with 3333 stored elements in Compressed Sparse Row format>

## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [11]:
x = churn.drop(['State', 'Phone', "Churn?"], axis=1)
y = churn["Churn?"]

## Séparation des données

Pour la séparation, on utilise la fonction `train_test_split()` de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type `test_size` :

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train.head(5)

,Account_Length,Area_Code,Int'l_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,Eve_Mins,Eve_Calls,Eve_Charge,Night_Mins,Night_Calls,Night_Charge,Intl_Mins,Intl_Calls,Intl_Charge,CustServ_Calls
3276,142,510,0,1,22,224.4,114,38.15,146.0,106,12.41,241.4,98,10.86,8.8,2,2.38,1
1123,11,408,0,1,24,131.5,98,22.36,230.2,111,19.57,283.7,87,12.77,10.0,3,2.70,2
1226,109,510,0,0,0,170.7,101,29.02,240.2,82,20.42,119.0,112,5.36,11.4,4,3.08,2
889,103,415,0,0,0,198.5,112,33.75,42.5,90,3.61,179.2,124,8.06,12.4,5,3.35,0
111,103,415,0,0,0,185.0,117,31.45,223.3,94,18.98,222.8,91,10.03,12.6,2,3.40,2


In [13]:
y_train.head(5)

3276    0
1123    0
1226    0
889     0
111     0
Name: Churn?, dtype: int64

In [14]:
y_train.value_counts(normalize=True)

0    0.850836
1    0.149164
Name: Churn?, dtype: float64

In [15]:
y_test.value_counts(normalize=True)

0    0.865
1    0.135
Name: Churn?, dtype: float64

Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

Ainsi les deux échantillons _train et _test ont la même distribution

In [17]:
y_train.value_counts(normalize=True)

0    0.855122
1    0.144878
Name: Churn?, dtype: float64

In [18]:
y_test.value_counts(normalize=True)

0    0.855
1    0.145
Name: Churn?, dtype: float64

## Le choix et l’ajustement de l’algorithme

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# ajouter d'autres modèles

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [20]:
modele_rf = RandomForestClassifier(n_estimators=100)
modele_knn = KNeighborsClassifier(n_neighbors=5)
# ajouter d'autres modèles

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données, avec la méthode `fit(x, y)` :

In [21]:
modele_rf.fit(x_train, y_train)
modele_knn.fit(x_train, y_train)
# ajouter d'autres modèles
pass

Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [22]:
[(k, v) for k, v in modele_knn.__dict__.items() if k.endswith('_')]

[('outputs_2d_', False),
 ('classes_', array([0, 1])),
 ('effective_metric_params_', {}),
 ('effective_metric_', 'euclidean')]

Ce qui va nous intéresser avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode `predict(x)` :

In [23]:
y_predict_rf = modele_rf.predict(x_test)
y_predict_knn = modele_knn.predict(x_test)
# ajouter d'autres modèles

On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module **metrics** de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [24]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test, y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test, y_predict_knn)
print("Pourcentage de bien classés pour le modèle RF : %.3f" % (accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" % (accuracy_modele_knn))

recall_modele_rf = recall_score(y_test, y_predict_rf)
recall_modele_knn = recall_score(y_test, y_predict_knn)
print("Rappel pour le modèle RF : %.3f" % (recall_modele_rf))
print("Rappel pour le modèle kNN :%.3f" % (recall_modele_knn))


Pourcentage de bien classés pour le modèle RF : 0.958
Pourcentage de bien classés pour le modèle kNN :0.872
Rappel pour le modèle RF : 0.738
Rappel pour le modèle kNN :0.228


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf = confusion_matrix(y_test, y_predict_rf)
confusion_matrix_knn = confusion_matrix(y_test, y_predict_knn)
print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")


Matrice de confusion pour le modèle RF :
[[851   4]
 [ 38 107]]
Matrice de confusion pour le modèle kNN :
[[839  16]
 [112  33]]


## Le rappel (recall), la précision et le f1-score

Scikit-Learn possède des fonctions pour chacun de ces indicateurs, mais il peut
être intéressant d’utiliser une autre fonction qui les affiche pour chaque classe :

tp / (tp + fn)

In [26]:
from sklearn.metrics import classification_report
print("Rapport pour le modèle RF :",
      classification_report(y_test,y_predict_rf), sep="\n")

Rapport pour le modèle RF :
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       855
           1       0.96      0.74      0.84       145

   micro avg       0.96      0.96      0.96      1000
   macro avg       0.96      0.87      0.91      1000
weighted avg       0.96      0.96      0.96      1000



In [27]:
print("Rapport pour le modèle kNN :",
      classification_report(y_test,y_predict_knn), sep="\n")

Rapport pour le modèle kNN :
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       855
           1       0.67      0.23      0.34       145

   micro avg       0.87      0.87      0.87      1000
   macro avg       0.78      0.60      0.63      1000
weighted avg       0.85      0.87      0.84      1000



In [28]:
# ajouter le 3ème modèle

## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [29]:
from sklearn.metrics import roc_auc_score
auc_modele_rf = roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn = roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :", auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :", auc_modele_knn)
# ajouter le 3ème modèle

Aire sous la courbe ROC pour le modèle RF : 0.9222141560798548
Aire sous la courbe ROC pour le modèle kNN : 0.6797983464408146


## La validation croisée
Jusqu’ici nous avons utilisé des indicateurs basés sur une seule occurrence de test. Ceci veut dire qu’on ne teste notre modèle que sur un seul échantillon.

Une approche alternative souvent utilisée est la validation croisée. Celle-ci est en fait basée sur la répétition de l’estimation et de la validation sur des données différentes.

Pour obtenir ce cv-score, on utilise :

In [30]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(modele_rf, x_train, y_train, cv=5, scoring='roc_auc')
scores_knn = cross_val_score(modele_knn, x_train, y_train, cv=5, scoring='roc_auc')

print("AUC pour RF : %.3f (+/- %.3f)" % (scores_rf.mean(), scores_rf.std() * 2))
print("AUC pour kNN : %.3f (+/- %.3f)" % (scores_knn.mean(),scores_knn.std() * 2))
# ajouter le 3ème modèle

AUC pour RF : 0.910 (+/- 0.058)
AUC pour kNN : 0.664 (+/- 0.029)


## L’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [31]:
from sklearn.model_selection import GridSearchCV

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [32]:
dico_param = {"max_depth": [3, 5, 7, 10, 25], "n_estimators": [10, 20, 50, 100]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [33]:
recherche_hyper = GridSearchCV(RandomForestClassifier(), 
                               dico_param, 
                               scoring="accuracy", cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [34]:
recherche_hyper.fit(x_train, y_train)
pass

In [35]:
print(recherche_hyper.best_params_)

{'max_depth': 25, 'n_estimators': 100}


In [36]:
print(recherche_hyper.best_score_)

0.9515645092156022


**Exercice :**

Effectuez le même travail avec GBM

## La construction d’un pipeline de traitement

Bien souvent vous allez être amené à enchaîner des traitements sur des données.
On peut bien sûr développer son code de manière à suivre les étapes une à une mais il est souvent plus intéressant de créer des suites de traitements automatisées avec Scikit-Learn. 

Ces suites de traitements sont appelées pipeline. Ils simplifieront votre code et permettront de passer en production simplement.

Ainsi, on va pouvoir faire une analyse en composantes principales suivie d’un
algorithme de plus proches voisins directement dans un pipeline :

In [37]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

In [38]:
acp = PCA(n_components=8)
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
pipe = Pipeline(steps=[("acp", acp), ("knn", knn)])
pipe2 = Pipeline(steps=[("acp", acp), ("rf", rf)])

pipe.fit(x_train, y_train)
pipe2.fit(x_train, y_train)
pass

On a ainsi enchaîné deux traitements. Si on cherche des sorties liées à chacune
des étapes, on pourra le faire simplement. Par exemple, si l’objectif est d’extraire la part de variances expliquées par les composantes de l’analyse en composantes principales, on fera :

In [39]:
pipe.named_steps["acp"].explained_variance_ratio_

array([0.23665287, 0.20455869, 0.19211346, 0.138076  , 0.12114616,
       0.03151708, 0.03106096, 0.02872042])

## Trouver la meilleure combinaison d’hyperparamètres dans un pipeline

Essayons de trouver la meilleure combinaison d’hyperparamètres dans un pipeline.
Dans le cadre de cet exemple, nous utiliserons les SVM (support vector machines,
également appelés séparateurs à vaste marge ou machines à vecteurs de support).
Ce sont des méthodes assez complexes dans leur principe mais simples dans leur
mise en oeuvre.

Les hyperparamètres d’un modèle de SVM sont assez nombreux, les plus importants
étant :
* le noyau choisi (linéaire, polynomial, sigmoïd, RBF…),
* les paramètres de ces noyaux (le degré pour le cas polynomial, gamma…),
* le C pour la marge floue.

Dans notre exemple, on utilisera la fonction `make_pipeline` qui est équivalente
à la classe précédente avec quelques simplifications :

In [40]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# construction du pipeline basé sur deux approches
mon_pipe = make_pipeline(PCA(), SVC(probability=True))

# construction du dictionnaire des paramètres
# (attention utilisation de __)
param_grid = dict(pca__n_components=[5, 10, x_train.shape[1]],
                  svc__C=[1, 10, 100, 1000],
                  svc__kernel=['sigmoid', 'rbf'],
                  svc__gamma=[0.001, 0.0001])

# on construit l’objet GridSearch et on estime les hyper-paramètres
# par validation croisée
grid_search_mon_pipe = GridSearchCV(mon_pipe, param_grid=param_grid, scoring="roc_auc", cv=4)

In [41]:
grid_search_mon_pipe.fit(x_train, y_train)
pass

In [42]:
# la meilleure combinaisons de paramètres est :
grid_search_mon_pipe.best_params_

{'pca__n_components': 18,
 'svc__C': 1,
 'svc__gamma': 0.0001,
 'svc__kernel': 'rbf'}

In [43]:
grid_search_mon_pipe.best_score_

0.7439860431757243

Les meilleurs hyperparamètres obtenus en utilisant l’aire sous la courbe ROC sont
la combinaison C = 10, gamma = 0.0001, un noyau RBF pour les SVM et dix composantes
pour notre analyse en composantes principales.

Dans ce code, on définit les hyperparamètres associés à une étape du pipeline
avec un double underscore (`__`).

L’utilisation des pipelines de Scikit-Learn va devenir rapidement une étape cruciale
de vos développements en Python.

## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [44]:
from sklearn.externals import joblib
joblib.dump(grid_search_mon_pipe, './data/modele_grid_pipe.pkl')

['./data/modele_grid_pipe.pkl']

Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [45]:
from sklearn.externals import joblib
grid_search_mon_pipe = joblib.load('./data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
grid_search_mon_pipe.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.